In [1]:
import pandas as pd
import os
import numpy as np
import random

base_path : str = os.path.dirname(os.getcwd())
CSV_PATH  : str = base_path + '\\csv'
SRC_PATH  : str = base_path + '\\src'
JSON_PATH : str = base_path + '\\json'

In [2]:
user_rating = pd.read_json(JSON_PATH + r'\USER_RATING_DATA.json')
user_rating

,USER_ID,RATED,NAN_RATED
0,5,"{'1': 2.0, '4': 2.0, '6': 3.0, '8': 4.0, '12':...","{'0': -1.0, '2': -1.0, '3': -1.0, '5': -1.0, '..."
1,12,"{'2': 2.0, '3': 5.0, '5': 4.0, '9': 1.0, '10':...","{'0': -1.0, '1': -1.0, '4': -1.0, '6': -1.0, '..."
2,19,"{'0': 5.0, '1': 2.0, '5': 2.0, '6': 5.0, '10':...","{'2': -1.0, '3': -1.0, '4': -1.0, '7': -1.0, '..."
3,24,"{'1': 5.0, '4': 5.0, '5': 5.0, '6': 5.0, '8': ...","{'0': -1.0, '2': -1.0, '3': -1.0, '7': -1.0, '..."
4,31,"{'1': 0.0, '6': 3.0, '7': 1.0, '10': 0.0, '11'...","{'0': -1.0, '2': -1.0, '3': -1.0, '4': -1.0, '..."
...,...,...,...
353,1796,"{'0': 3.0, '1': 1.0, '3': 3.0, '6': 0.0, '7': ...","{'2': -1.0, '4': -1.0, '5': -1.0, '8': -1.0, '..."
354,1804,"{'1': 3.0, '2': 2.0, '3': 4.0, '9': 3.0, '10':...","{'0': -1.0, '4': -1.0, '5': -1.0, '6': -1.0, '..."
355,1813,"{'0': 3.0, '1': 5.0, '2': 5.0, '4': 3.0, '5': ...","{'3': -1.0, '7': -1.0, '8': -1.0, '9': -1.0, '..."
356,1815,"{'2': 4.0, '3': 3.0, '4': 2.0, '5': 1.0, '48':...","{'0': -1.0, '1': -1.0, '6': -1.0, '7': -1.0, '..."


In [3]:
USER_HYPER_PARAMS = pd.read_json(JSON_PATH + r'\USER_HYPER_PARAMS.json')

In [4]:
movie_distance = pd.read_json(JSON_PATH + r'\movie_distance_graph.json')

In [10]:
def decision_tree(movie_id, all_distances, training_ids, rating_data, threshold) -> int:

    """
        Dla movie_id wyszukujemyt filmy, które oddalone są od movie_id o mniej niż THRESHOLD.
        Jeśli taki jest, to zapamiętujemy jego ocenę, a potem ze wszystkich wyciątgamy średnią.
    """
    
    decisions: list = []

    for rated_movie in training_ids:

        feature_counter: int = 0

        for feat_dist_i, feature_distance in enumerate(all_distances[movie_id][int(rated_movie)]):

            if feature_distance < threshold[feat_dist_i]:

                feature_counter += 1

        if feature_counter == 5:

            decisions.append(rating_data[str(rated_movie)])


    return np.ceil(np.average(decisions)) if len(decisions) > 0 else None

def test_user(user, best_weights, predict_ids, train_ids) -> tuple[list, list]:

    ratings: list = []
    movies:  list = []

    for predict_movie in predict_ids:

        output = decision_tree(movie_id=int(predict_movie), all_distances=movie_distance, training_ids=train_ids, rating_data=user_rating["RATED"][user], threshold=best_weights)
        ratings.append(output)
        movies.append(predict_movie)

    return (movies, ratings)

In [9]:
user = 0
user_id :    int  = user_rating['USER_ID'][user]
best_weight: list = USER_HYPER_PARAMS[user_id]
preditc_ids: list = list(user_rating['NAN_RATED'][user].keys())
train_ids:   list = list(user_rating['RATED'][user].keys())

print(f'Train: {train_ids}, Predict: {preditc_ids}')

Train: ['1', '4', '6', '8', '12', '32', '37', '38', '39', '40', '44', '46', '48', '52', '54', '59', '60', '61', '62', '63', '64', '68', '71', '72', '73', '74', '75', '78', '79', '80', '84', '87', '88', '90', '91', '99', '101', '103', '104', '110', '114', '115', '119', '121', '124', '125', '126', '127', '129', '130', '131', '132', '134', '135', '137', '140', '141', '145', '146', '148', '150', '151', '152', '153', '159', '162', '163', '164', '165', '166', '167', '168', '169', '171', '172', '174', '176', '177', '180', '181', '182', '185', '186', '189', '190', '192', '193', '194', '195', '196'], Predict: ['0', '2', '3', '5', '7', '9', '10', '11', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '33', '34', '35', '36', '41', '42', '43', '45', '47', '49', '50', '51', '53', '55', '56', '57', '58', '65', '66', '67', '69', '70', '76', '77', '81', '82', '83', '85', '86', '89', '92', '93', '94', '95', '96', '97', '98', '100', '102',

In [ ]:
from json import dump, load
user_rating_data_predicted : list = []
user_rating_data_movies: list = []

# NUM_OF_USERS : int = len(user_rating)
NUM_OF_USERS : int = 1

for user in range(NUM_OF_USERS):
    
    user_id :    int  = user_rating['USER_ID'][user]
    best_weight: list = USER_HYPER_PARAMS[user_id]
    preditc_ids: list = list(user_rating['NAN_RATED'][user].keys())
    train_ids:   list = list(user_rating['RATED'][user].keys())

    returned_movies, returned_reviews = test_user(user=user, best_weights=best_weight, predict_ids=preditc_ids, train_ids=train_ids)
    
    user_rating_data_movies.append(returned_movies)
    user_rating_data_predicted.append(returned_reviews)

# for i in range(NUM_OF_USERS):
#     print(f"User {user_rating['USER_ID'][i]} - Movies: {user_rating_data_movies[i]}, Reviews: {user_rating_data_predicted[i]}")


user_test_data = {
    user_rating['USER_ID'][i]: {
       'MOVIES' : [str(int(movie_id) + 1) for movie_id in user_rating_data_movies[i]],
       'RATINGS' : user_rating_data_predicted[i]
    }
    for i in range(NUM_OF_USERS)
}

user_test_data_df = pd.DataFrame(user_test_data)
user_test_data_df.to_json(JSON_PATH + '\\USER_PREDICTED_RATINGS.json', indent=4)

with open(JSON_PATH + '\\USER_PREDICTED_RATINGS.json', 'r') as f:
    data = load(f)

user_test_data = {}
for user_id, ratings_data in data.items():
    movies = ratings_data['MOVIES']
    ratings = ratings_data['RATINGS']
    
    user_test_data[user_id] = {movie: rating for movie, rating in zip(movies, ratings)}


with open(JSON_PATH + '\\USER_PREDICTED_RATINGS_TRANSFORMED.json', 'w') as f:
    dump(user_test_data, f, indent=4)

Train: ['1', '4', '6', '8', '12', '32', '37', '38', '39', '40', '44', '46', '48', '52', '54', '59', '60', '61', '62', '63', '64', '68', '71', '72', '73', '74', '75', '78', '79', '80', '84', '87', '88', '90', '91', '99', '101', '103', '104', '110', '114', '115', '119', '121', '124', '125', '126', '127', '129', '130', '131', '132', '134', '135', '137', '140', '141', '145', '146', '148', '150', '151', '152', '153', '159', '162', '163', '164', '165', '166', '167', '168', '169', '171', '172', '174', '176', '177', '180', '181', '182', '185', '186', '189', '190', '192', '193', '194', '195', '196'], Predict: ['0', '2', '3', '5', '7', '9', '10', '11', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '33', '34', '35', '36', '41', '42', '43', '45', '47', '49', '50', '51', '53', '55', '56', '57', '58', '65', '66', '67', '69', '70', '76', '77', '81', '82', '83', '85', '86', '89', '92', '93', '94', '95', '96', '97', '98', '100', '102',